In [1]:
import numpy as np
import pandas as pd
import re, json, csv, requests, time, glob, tqdm
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
def scrape(html):
    soup = BeautifulSoup(html, "html.parser")
    json_data = json.loads(soup.find_all("script", type="application/ld+json")[-1].text)
    title = json_data['headline']
    date = json_data['datePublished']
    date_m = json_data['dateModified']
    genre = json_data.get('genre', [])
    keywords = json_data.get('keywords', [])
    article = soup.find('div', id="news_textbody").text
    url_normal = 'https:' + soup.find('meta', property="og:url").get('content').rsplit('https:')[-1]
    if soup.find_all('div', id="news_textmore") != []:
        for textmore in soup.find_all('div', id="news_textmore"):
            article += ('\n' + textmore.text)
    if soup.find_all('div', class_="news_add") != []:
        for newsadd in soup.find_all('div', class_="news_add"):
            if newsadd.h3 != None:
                newsadd.h3.extract()
            article += ('\n' + newsadd.text)
            
    return {
        'id':url_normal.split('/')[-1].split('.html')[0],
        'title':title,
        'article':article.strip(),
        'genre':genre,
        'keywords':keywords,
        'url':url_normal,
        'datePublished':date,
        'dateModified':date_m
    }

def js(dic):
    with open('nhkweb.json', 'r', encoding='utf-8') as f:
        data = json.load(f)
    with open('nhkweb.json', 'w', encoding='utf-8') as f:
        if dic not in data:
            data.append(dic)
        data = sorted(data, key=lambda x:x['id'])
        json.dump(data, f, indent=4, ensure_ascii=False)
        
easy = pd.read_json('nhkwebeasy.json')

In [ ]:
driver = webdriver.Chrome()

In [ ]:
for nhkurl in tqdm.tqdm(htmls):
    driver.get(f'https://web.archive.org/web/2019*/{nhkurl}')
    time.sleep(10)
    html = str(driver.page_source.encode('utf-8'))
    snap = re.search(r'between <a href="(.+?)">', html)
    if not snap:
        print(nhkurl)
        continue
    driver.get('https://web.archive.org' + snap.group(1))
    time.sleep(30)
    html = driver.page_source.encode('utf-8')
    dic = scrape(html)
    js(dic)